In [5]:
import pandas as pd
import numpy as np

In [8]:
file_path = "/content/drive/MyDrive/Colab Notebooks/Sample - Superstore.csv" # Updated path
df = pd.read_csv(file_path, encoding='latin1')

print("Rows, Columns:", df.shape)
df.head()

Rows, Columns: (9994, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [9]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [10]:
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")

print("Missing Order Date after conversion:", df["Order Date"].isna().sum())
df[["Order Date"]].head()

Missing Order Date after conversion: 0


,Order Date
0,2016-11-08
1,2016-11-08
2,2016-06-12
3,2015-10-11
4,2015-10-11


In [11]:
df = df.dropna(subset=["Order Date", "Sales", "Profit", "Category", "Sub-Category", "Product Name"])

print("After cleaning:", df.shape)

After cleaning: (9994, 21)


In [12]:
df["profit_margin"] = df["Profit"] / df["Sales"].replace(0, np.nan)

df[["Sales", "Profit", "profit_margin"]].head()

,Sales,Profit,profit_margin
0,261.9600,41.9136,0.1600
1,731.9400,219.5820,0.3000
2,14.6200,6.8714,0.4700
3,957.5775,-383.0310,-0.4000
4,22.3680,2.5164,0.1125


In [13]:
category_summary = df.groupby("Category").agg(
    total_sales=("Sales", "sum"),
    total_profit=("Profit", "sum"),
    avg_profit_margin=("profit_margin", "mean")
).reset_index()

category_summary["profit_margin_pct"] = category_summary["avg_profit_margin"] * 100
category_summary = category_summary.sort_values("total_profit")

category_summary

,Category,total_sales,total_profit,avg_profit_margin,profit_margin_pct
0,Furniture,741999.7953,18451.2728,0.038784,3.878353
1,Office Supplies,719047.0320,122490.8008,0.138030,13.803029
2,Technology,836154.0330,145454.9481,0.156138,15.613805


In [14]:
subcategory_summary = df.groupby(["Category", "Sub-Category"]).agg(
    total_sales=("Sales", "sum"),
    total_profit=("Profit", "sum"),
    avg_profit_margin=("profit_margin", "mean")
).reset_index()

subcategory_summary["profit_margin_pct"] = subcategory_summary["avg_profit_margin"] * 100
subcategory_summary = subcategory_summary.sort_values("total_profit")

subcategory_summary.head(15)

,Category,Sub-Category,total_sales,total_profit,avg_profit_margin,profit_margin_pct
3,Furniture,Tables,206965.5320,-17725.4811,-0.147727,-14.772653
0,Furniture,Bookcases,114879.9963,-3472.5560,-0.126640,-12.664007
12,Office Supplies,Supplies,46673.5380,-1189.0995,0.112039,11.203947
8,Office Supplies,Fasteners,3024.2800,949.5182,0.299171,29.917051
15,Technology,Machines,189238.6310,3384.7569,-0.072026,-7.202622
9,Office Supplies,Labels,12486.3120,5546.2540,0.429663,42.966346
5,Office Supplies,Art,27118.7920,6527.7870,0.251646,25.164573
7,Office Supplies,Envelopes,16476.4020,6964.1767,0.423140,42.313976
2,Furniture,Furnishings,91705.1640,13059.1436,0.137066,13.706635
4,Office Supplies,Appliances,107532.1610,18138.0054,-0.156869,-15.686934


In [15]:
df = df.sort_values(["Product Name", "Order Date"])

df["days_since_prev_order"] = df.groupby("Product Name")["Order Date"].diff().dt.days

df[["Product Name", "Order Date", "days_since_prev_order"]].head(10)

,Product Name,Order Date,days_since_prev_order
7321,"""While you Were Out"" Message Book, One Form pe...",2017-09-03,NaN
1708,"""While you Were Out"" Message Book, One Form pe...",2017-10-30,57.0
3782,"""While you Were Out"" Message Book, One Form pe...",2017-11-13,14.0
2585,"#10 Gummed Flap White Envelopes, 100/Box",2015-11-03,NaN
3741,"#10 Gummed Flap White Envelopes, 100/Box",2016-01-15,73.0
355,"#10 Gummed Flap White Envelopes, 100/Box",2016-04-08,84.0
7827,"#10 Gummed Flap White Envelopes, 100/Box",2017-10-31,571.0
3672,#10 Self-Seal White Envelopes,2016-04-08,NaN
4640,#10 Self-Seal White Envelopes,2016-09-11,156.0
3695,#10 Self-Seal White Envelopes,2016-10-15,34.0


In [16]:
product_metrics = df.groupby("Product Name").agg(
    avg_days_between_orders=("days_since_prev_order", "mean"),
    total_sales=("Sales", "sum"),
    total_profit=("Profit", "sum"),
    avg_margin=("profit_margin", "mean")
).reset_index()

# products ordered only once → NaN gap; set them to max (slow-moving)
product_metrics["avg_days_between_orders"] = product_metrics["avg_days_between_orders"].fillna(
    product_metrics["avg_days_between_orders"].max()
)

product_metrics.head()

,Product Name,avg_days_between_orders,total_sales,total_profit,avg_margin
0,"""While you Were Out"" Message Book, One Form pe...",35.500000,25.228,10.3880,0.416667
1,"#10 Gummed Flap White Envelopes, 100/Box",242.666667,41.300,16.7678,0.392500
2,#10 Self-Seal White Envelopes,173.666667,108.682,52.1230,0.458125
3,"#10 White Business Envelopes,4 1/8 x 9 1/2",167.500000,488.904,223.1408,0.432143
4,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",143.000000,286.672,115.2806,0.417000


In [17]:
corr = product_metrics[["avg_days_between_orders", "total_profit", "avg_margin"]].corr()
corr

,avg_days_between_orders,total_profit,avg_margin
avg_days_between_orders,1.000000,-0.067566,-0.004572
total_profit,-0.067566,1.000000,0.181642
avg_margin,-0.004572,0.181642,1.000000


In [18]:
threshold = product_metrics["avg_days_between_orders"].quantile(0.75)

slow_movers = product_metrics[
    product_metrics["avg_days_between_orders"] >= threshold
].sort_values("total_profit")

slow_movers.head(15)

,Product Name,avg_days_between_orders,total_sales,total_profit,avg_margin
476,Cubify CubeX 3D Printer Triple Head Print,1324.000000,7999.980,-3839.9904,-0.480000
444,Cisco TelePresence System EX90 Videoconferenci...,1324.000000,22638.480,-1811.0784,-0.080000
285,Balt Solid Wood Round Tables,436.333333,6518.754,-1201.0581,-0.213333
364,BoxOffice By Design Rectangular and Half-Moon ...,613.000000,1706.250,-1148.4375,-0.687273
603,Epson TM-T88V Direct Thermal Printer - Monochr...,1168.000000,1212.705,-1057.2300,-0.825000
1838,Zebra GK420t Direct Thermal/Thermal Transfer P...,1324.000000,703.710,-938.2800,-1.333333
17,3.6 Cubic Foot Counter Height Office Refrigerator,342.500000,2946.200,-872.0752,-1.484000
836,Hon 94000 Series Round Tables,328.500000,7404.500,-681.2140,-0.122333
321,Bevis Rectangular Conference Tables,361.666667,1007.262,-586.8396,-0.610000
986,Lexmark MarkNet N8150 Wireless Print Server,651.000000,723.510,-506.4570,-0.700000


In [19]:
category_summary.to_csv("category_profit_python.csv", index=False)
subcategory_summary.to_csv("subcategory_profit_python.csv", index=False)
product_metrics.to_csv("product_metrics.csv", index=False)
slow_movers.to_csv("slow_movers.csv", index=False)

print("Saved: category_profit_python.csv, subcategory_profit_python.csv, product_metrics.csv, slow_movers.csv")

Saved: category_profit_python.csv, subcategory_profit_python.csv, product_metrics.csv, slow_movers.csv
